In [1]:
!pip3 install pandas h3 sqlalchemy boto3

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import os

os.path.isfile("./kubernetes-oracle.pem")

True

In [13]:
print("starting...")



# -*- coding: utf-8 -*-
"""part_1_mobility_script_generic(new_dataset)_new (2).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1dGNHBaJAS1w2hsZw4c_Kx4bFdQ0Kzdg_
"""

print("starting...")

from pyspark.sql.window import Window 
import pyspark
from pyspark import SparkContext

from pyspark.sql import functions as F
import pyspark.sql.functions as func


from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import col, window
import pyspark.sql.functions as func
from pyspark.sql.functions import desc
import datetime, time
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as sf

import datetime as dt
import numpy as np
import os

from multiprocessing.pool import Pool
import pickle
import pandas as pd

from h3 import h3


from math import radians, cos, sin, asin, sqrt
from pyspark.sql.functions import lit
import json

from datetime import datetime, date, timedelta

#spark.conf.set("fs.azure.account.key.iasocialdata.dfs.core.windows.net",")
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")  #### Enable Arrow-based columnar data transfers

#sc.install_pypi_package("boto3==1.16.35")
#sc.install_pypi_package("SqlAlchemy")
#sc.install_pypi_package("pymysql")

# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("S3CSVRead").getOrCreate()

#accessKeyId=""
#secretAccessKey=""
#access_key = accessKeyId
#secret_key = secretAccessKey
#sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
#sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

# If you are using Auto Loader file notification mode to load files, provide the AWS Region ID.
#aws_region = "us-east-1"
#sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

from pyspark.sql import SparkSession
import sys
import boto3
from io import BytesIO

from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.functions import round, col
from dateutil import tz
import sqlalchemy as db

import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# spark.conf.set('fs.oci.client.auth.tenantId', "ocid1.tenancy.oc1..")
# spark.conf.set('fs.oci.client.auth.userId', "ocid1.user.oc1..")
# spark.conf.set('fs.oci.client.auth.fingerprint', "ec:8f:0e:14:e0:31:cd:e4:d6:03:04:aa:aa:94:f9:a9")
# spark.conf.set('fs.oci.client.auth.pemfilepath', "~/.oci/oci_api_key")
# spark.conf.set('fs.oci.client.auth.tenantId.region', "us-ashburn-1")
# spark.conf.set('fs.oci.client.hostname', "https://objectstorage.us-ashburn-1.oraclecloud.com")



starting...
starting...


In [14]:
####Arguments
date_start = "2021-12-05"#dbutils.widgets.get('date_start')#"2021-12-09"
country = "IND" #dbutils.widgets.get('country')#"ARE"
unique_token = "PROD_API_2021-12-10_{0}_{1}".format(date_start,country)
####Arguments

token = requestId = "{}_{}".format(unique_token, date_start)

#days = (datetime.now() - dt.datetime.strptime(date_start, "%Y-%m-%d")).days
str_arg = ""
tminus = dt.datetime.strptime(date_start, "%Y-%m-%d")#date.today() - timedelta(days=int(38))# 2021-04-12
print(tminus)
date_string = tminus.strftime('%d')
month_string = tminus.strftime('%m')
year_string  = tminus.strftime('%Y')
print(date_string)

str_arg  = str_arg+"start time="+str(datetime.now())+"\n"

log_str = ""

2021-12-05 00:00:00
05


In [15]:
def createLogs():
    
    global log_str 

    try:
        f = BytesIO()

        #session = boto3.Session(
        #aws_access_key_id=accessKeyId,
        #aws_secret_access_key=secretAccessKey)
        #s3_client = session.client("s3")
    
        log_str ="request_id="+requestId+";notebook_name=Date wise Split for Vendor data;start_time="+str(time.time())+"\n request_id="+requestId+";message=Started Date wise split \n"
    #analytics_logs = str(f.getvalue().decode()) +  str(text_string)
    # Method 1: Object.put()
        #s3 = session.resource('s3')
        #logs_path = 'logs/{0}/{1}_pre-process.txt'.format(str(tminus.strftime('%Y-%m-%d')), country)
        #object = s3.Object('ia-mobility-logs', logs_path)
        #object.put(Body=more_binary_data)
    except Exception as e:
        print(e)
        
createLogs()

def writeToLogs(str_arg):

    global log_str

    try:
        f = BytesIO()

        #session = boto3.Session(
        #aws_access_key_id=accessKeyId,
        #aws_secret_access_key=secretAccessKey)
    
        #s3_client = session.client("s3")
        #logs_path = 'logs/{0}/{1}_pre-process.txt'.format(str(tminus.strftime('%Y-%m-%d')), country)
    
        #s3_client.download_fileobj('ia-mobility-logs', logs_path, f)
        #f.seek(0)
    
        log_str =  log_str +"\n"+"notebook_name=Date wise Split for Vendor data;" +  str(str_arg)
        #analytics_logs = str(f.getvalue().decode()) +  str(text_string)
        # Method 1: Object.put()
        #s3 = session.resource('s3')
        #logs_path = 'logs/{0}/{1}_pre-process.txt'.format(str(tminus.strftime('%Y-%m-%d')), country)
        #object = s3.Object('ia-mobility-logs', logs_path)
        #object.put(Body=more_binary_data)
        return "completed"
    except Exception as e:
        print(e)
        return e

In [16]:
def getFiles(date_val, month_val):
    try:
        paths = "oci://ia-location-data@bmmp5bv7olp2/{year_string}/{month_string}/{date_string}/"
        #f"oci://ia-location-data@bmmp5bv7olp2/{year_string}/"+month_string+"/"+date_string+"/"
        #paths = f"oci://ia-location-data@bmmp5bv7olp2/{year_string}/"+month_string+"/"+date_string+"/*.parquet"
        print(paths)
        df = spark.read.parquet(paths)
        print(writeToLogs("request_id="+requestId+"message=File checking started. File found at following location;file_path="+paths))
    except Exception as e:
        print(e)
        paths = f"oci://ia-location-data@bmmp5bv7olp2/{year_string}/"+month_string+"/"+date_string+"/*.parquet"
        df = spark.read.parquet(paths)
        print("file not found")
        print(writeToLogs("request_id="+requestId+"message=File checking started. File found at following location;file_path="+paths))

    print(paths)
    return paths

In [17]:
paths = getFiles(date_string,month_string)
str_arg = str_arg + paths +"\n"

start_time = time.time()
df = spark.read.parquet(paths)
log_str= "request_id="+requestId+";task_id=1.0;task_name=Read file from input;task_start_time="+str(start_time)+";task_end_time="+str(time.time())+\
         ";task_time_taken="+str(time.time()-start_time)+";status=suceessful;message=ok" 
print(writeToLogs(log_str))

start_time = time.time()
df.show(3)
df0 = df.filter(df.country == country)

df1 = df0.filter(df0.latitude != "0.0")
df1 = df1.filter(df1.longitude != "0.0")

log_str= "request_id="+requestId+";task_id=1.1;task_name=Remove incorrect lat, long;task_start_time="+str(start_time)+";task_end_time="+str(time.time())+\
         ";task_time_taken="+str(time.time()-start_time)+";status=suceessful;message=ok" 
writeToLogs(log_str)


start_time = time.time()
#df2 = df1.withColumn("timestamp", df["timestamp"] / 1000)
df2 = df1.withColumn("timestamp", F.from_unixtime(F.col("timestamp"), 'yyyy-MM-dd HH:mm:ss').cast("timestamp"))
#df2.show()

oci://ia-location-data@bmmp5bv7olp2/{year_string}/{month_string}/{date_string}/
Path does not exist: oci://ia-location-data@bmmp5bv7olp2/{year_string}/{month_string}/{date_string}
file not found
completed
oci://ia-location-data@bmmp5bv7olp2/2021/12/05/*.parquet
completed
+--------------------+-------+---------+---------+-------------------+-----------------+--------+--------------+----+----------+-------+----------+---------+--------+--------+---------------+---------------+-------+-----+---------+----------+-------------+----------+
|                maid|id_type| latitude|longitude|horizontal_accuracy|vertical_accuracy|altitude|          ipv4|ipv6|user_agent|country|state_hasc|city_hasc|postcode| geohash|           hex8|           hex9|heading|speed|wifi_ssid|wifi_bssid|      carrier| timestamp|
+--------------------+-------+---------+---------+-------------------+-----------------+--------+--------------+----+----------+-------+----------+---------+--------+--------+---------------+-

In [8]:
df2.show(3)

+--------------------+-------+---------+---------+-------------------+-----------------+--------+--------------+----+----------+-------+----------+---------+--------+--------+---------------+---------------+-------+-----+---------+----------+-------------+-------------------+
|                maid|id_type| latitude|longitude|horizontal_accuracy|vertical_accuracy|altitude|          ipv4|ipv6|user_agent|country|state_hasc|city_hasc|postcode| geohash|           hex8|           hex9|heading|speed|wifi_ssid|wifi_bssid|      carrier|          timestamp|
+--------------------+-------+---------+---------+-------------------+-----------------+--------+--------------+----+----------+-------+----------+---------+--------+--------+---------------+---------------+-------+-----+---------+----------+-------------+-------------------+
|81e78497-a6c2-464...|      1| 12.99099|77.663438|                7.0|             null|    null|157.45.175.244|null|      null|    IND|     IN.KA| IN.KA.BN|  560093|tdr

In [10]:
from datetime import datetime
from dateutil import tz

def return_time(time):
  from_zone = tz.gettz('UTC')

  if(country == "IND"):
    to_zone = tz.gettz('Asia/Kolkata')
  elif(country == "ARE"):
    to_zone = tz.gettz('Asia/Dubai')
  elif(country == "MEX"):
    to_zone = tz.gettz('America/Mexico_City')
  else:
    to_zone = tz.gettz('UTC')
    writeToLogs("New Country")   

  utc = datetime.strptime(str(time), '%Y-%m-%d %H:%M:%S')
  utc = utc.replace(tzinfo=from_zone)

  central = utc.astimezone(to_zone)      
  return [str(central)[:-6]]

schema = StructType([
    StructField("time", StringType(), False)
])

loc_test = udf(lambda a: return_time(a), schema)

#df3 = df2.select('maid', 'id_type', 'latitude', 'longitude', 'horizontal_accuracy', 'vertical_accuracy', 'altitude', 'ipv4', 'ipv6', 'user_agent', 'country', 'state_hasc', 'city_hasc', 'postcode', 'geohash', 'hex8', 'hex9', #'heading', 'speed', 'wifi_ssid', 'wifi_bssid', 'carrier', loc_test('timestamp').alias('indexTime'))

#df3 = df3.select('maid', 'id_type', 'latitude', 'longitude', 'horizontal_accuracy', 'vertical_accuracy', 'altitude', 'ipv4', 'ipv6', 'user_agent', 'country', 'state_hasc', 'city_hasc', 'postcode', 'geohash', 'hex8', 'hex9', #'heading', 'speed', 'wifi_ssid', 'wifi_bssid', 'carrier', 'indexTime.time')

#df3 = df3.selectExpr('maid as maid', 'id_type as id_type', 'latitude as latitude', 'longitude as longitude', 'horizontal_accuracy as horizontal_accuracy', 'vertical_accuracy as vertical_accuracy', 'altitude as altitude', #'ipv4 as ipv4', 'ipv6 as ipv6', 'user_agent as user_agent', 'country as country', 'state_hasc as state_hasc', 'city_hasc as city_hasc', 'postcode as postcode', 'geohash as geohash', 'hex8 as hex8', 'hex9 as hex9', 'heading #as heading', 'speed as speed', 'wifi_ssid as wifi_ssid', 'wifi_bssid as wifi_bssid', 'carrier as carrier', 'time as timestamp')

df3 = df2.withColumn("country", lit(country)).withColumn("converted_ts", when(col("country") == 'ARE', from_utc_timestamp(col("timestamp"), "Asia/Dubai")).otherwise(when(col("country") == 'IND', from_utc_timestamp(col("timestamp"), "Asia/Kolkata")).otherwise(from_utc_timestamp(col("timestamp"), "UTC"))))

df3 = df3.withColumn('date', F.to_date(col('timestamp')).cast("date"))

from pyspark.sql.functions import *
df3 = df3.withColumn("timestamp", to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss"))
# df3.show(3)


log_str= "request_id="+requestId+";task_id=1.2;task_name=Convert date time;task_start_time="+str(start_time)+";task_end_time="+str(time.time())+\
         ";task_time_taken="+str(time.time()-start_time)+";status=suceessful;message=ok" 
print(writeToLogs(log_str))

df3.show(3)

total_rows = df3.count()
print(total_rows)

total_valid_rows = df3.count()
print(total_valid_rows)

str_arg = str_arg + "total_rows="+str(total_rows)+"\n"+"total_valid_rows="+\
          str(total_valid_rows)+"\n"

if(total_rows == total_valid_rows):
    str_arg = str_arg + "rows count equal \n"
else: 
    str_arg = str_arg+"rows count not equal \n"

dates = df3.groupby('date').count()

#dates.sort('date').show()
percent_data_spill = dates.withColumn('percent', (col("count")/total_valid_rows)*100.0)
#percent_data_spill.show()

date_values = dates.select('date').collect()

def filterOutDate(date_val):
    df4 = df3.filter(df3.date == date_val)
    df4.show(3)

folder_date = tminus.strftime('%Y-%m-%d')

save_string = "oci://ia-location-data@bmmp5bv7olp2/lf_data/hdfs_data/Updated_LS/{0}/{1}".format(country,folder_date)
str_arg = str_arg+"files_path="+save_string+"\n"

start_time = time.time()
numPartitions = 800
newDF = df3#.repartition(numPartitions)
newDF.show(3)
log_str= "request_id="+requestId+";task_id=2.0;task_name=Repartition data;task_start_time="+str(start_time)+";task_end_time="+str(time.time())+\
         ";task_time_taken="+str(time.time()-start_time)+";status=suceessful;message=ok" 
print(writeToLogs(log_str))

completed
+--------------------+-------+---------+---------+-------------------+-----------------+--------+--------------+----+----------+-------+----------+---------+--------+--------+---------------+---------------+-------+-----+---------+----------+-------------+-------------------+-------------------+----------+
|                maid|id_type| latitude|longitude|horizontal_accuracy|vertical_accuracy|altitude|          ipv4|ipv6|user_agent|country|state_hasc|city_hasc|postcode| geohash|           hex8|           hex9|heading|speed|wifi_ssid|wifi_bssid|      carrier|          timestamp|       converted_ts|      date|
+--------------------+-------+---------+---------+-------------------+-----------------+--------+--------------+----+----------+-------+----------+---------+--------+--------+---------------+---------------+-------+-----+---------+----------+-------------+-------------------+-------------------+----------+
|81e78497-a6c2-464...|      1| 12.99099|77.663438|                

In [11]:
start_time = time.time()
try:
    newDF.write.partitionBy(['date']).mode("overwrite").format('delta').save(save_string)
    log_str= "request_id="+requestId+";task_id=2.1;task_name=Write to S3;task_start_time="+str(start_time)+";task_end_time="+str(time.time())+\
         ";task_time_taken="+str(time.time()-start_time)+";status=suceessful;message=ok" 
    writeToLogs(log_str)
except Exception as e:
    log_str= "request_id="+requestId+";task_id=2.1;task_name=Write to s3;task_start_time="+str(start_time)+";task_end_time="+str(time.time())+\
         ";task_time_taken="+str(time.time()-start_time)+";status=fail;message=error;error_message="+str(e) 
    writeToLogs(log_str)

analytics_save_path = "oci://ia-location-data@bmmp5bv7olp2/logs/LS/Analytics/"+str(tminus.strftime("%Y-%m-%d"))+".csv"


text_string=""

for x in percent_data_spill.sort('date').collect():
    text_string = text_string +"total_rows="+str(total_rows)+",total_valid_rows="+\
    str(total_valid_rows)+\
    ",date="+str(x[0])+",user_row="+str(x[1])+",percentage="+"{0:.2f}".format(x[2])+\
    ",received_date="+str(tminus.strftime('%Y-%m-%d'))+"\n"

print(text_string)

log_str= "request_id="+requestId+";task_id=2.2;task_name=File execution completed;task_start_time="+str(start_time)+";task_end_time="+str(time.time())+\
         ";task_time_taken="+str(time.time()-start_time)+";status=suceessful;message=ok"
writeToLogs(log_str) 


file_path =  'analytics/LS/{}/analysis.txt'.format(str(tminus.strftime('%Y-%m-%d')))



total_rows=293912304,total_valid_rows=293912304,date=2021-11-26,user_row=635054,percentage=0.22,received_date=2021-12-05
total_rows=293912304,total_valid_rows=293912304,date=2021-11-27,user_row=2363439,percentage=0.80,received_date=2021-12-05
total_rows=293912304,total_valid_rows=293912304,date=2021-11-28,user_row=17457258,percentage=5.94,received_date=2021-12-05
total_rows=293912304,total_valid_rows=293912304,date=2021-11-29,user_row=26649614,percentage=9.07,received_date=2021-12-05
total_rows=293912304,total_valid_rows=293912304,date=2021-11-30,user_row=36943362,percentage=12.57,received_date=2021-12-05
total_rows=293912304,total_valid_rows=293912304,date=2021-12-01,user_row=66425616,percentage=22.60,received_date=2021-12-05
total_rows=293912304,total_valid_rows=293912304,date=2021-12-02,user_row=56910079,percentage=19.36,received_date=2021-12-05
total_rows=293912304,total_valid_rows=293912304,date=2021-12-03,user_row=72166999,percentage=24.55,received_date=2021-12-05
total_rows=2939